In [ ]:
import json
import time
import pandas as pd
import requests
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
print('Bibliotecas importadas')

In [ ]:
SQLALCHEMY = os.environ.get('SQLALCHEMY_HOMOLOGACAO')

try:
    engine = create_engine(SQLALCHEMY)
    conn_hom = engine.connect()
    print('Conexão com banco de dados criada...')
except:
    print('Falha ao criar conexão com banco de dados.')

In [ ]:
print(SQLALCHEMY)

In [ ]:
dfpesquisa = pd.read_json("https://servicodados.ibge.gov.br/api/v3/agregados?nivel=N6")

In [ ]:
dfpesquisa

In [ ]:
df_dim_pesquisa = dfpesquisa[['id','nome']]

In [ ]:
df_dim_pesquisa

In [ ]:
list_pesquisas = {'CD'}

In [ ]:
list_pesquisas

In [ ]:
df_dim_agregados = pd.DataFrame(columns=['id','nome'])
for i, row in dfpesquisa.iterrows():
    if row['id'] in list_pesquisas:
        df_temp = pd.json_normalize(row['agregados'])
        df_temp['pesquisa'] = row['id']
        df_dim_agregados = pd.concat([df_dim_agregados,df_temp])
    

In [ ]:
df_dim_agregados.to_csv("dim_agregados_a_pesquisar.csv")

In [ ]:
df_dim_agregados

In [ ]:
lista_a_pesquisar = list(df_dim_agregados['id'])
lista_a_pesquisar

In [ ]:
municipios = [
    {"municipio": "Curionópolis", "codigo_ibge": "1502772"},
    {"municipio": "Parauapebas", "codigo_ibge": "1505536"},
    {"municipio": "Canaã dos Carajás", "codigo_ibge": "1502152"},
    {"municipio": "Eldorado do Carajás", "codigo_ibge": "1502954"},
    {"municipio": "Água Azul do Norte", "codigo_ibge": "1500347"}
]

def get_base(id):
    time.sleep(2)
    url = f"https://servicodados.ibge.gov.br/api/v3/agregados/{id}/periodos/-30/variaveis?localidades=N6[1505536]"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload, timeout=10)
    return response.json()

In [ ]:
def get_base(id, mun):
    retries = 3  # Número de tentativas
    delay = 2  # Tempo de espera entre tentativas
    for _ in range(retries):
        try:
            url = f"https://servicodados.ibge.gov.br/api/v3/agregados/{id}/periodos/-30/variaveis?localidades=N6[{mun}]"
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Lança uma exceção se a resposta indicar um erro HTTP
            return response.json()
        except (requests.RequestException, ValueError) as e:
            print(f"Erro ao fazer a requisição: {e}")
            time.sleep(delay)
            continue
    print(f"Falha nas tentativas de solicitação após várias tentativas. {id}")
    return "ERRO"  # Ou levante uma exceção, dependendo do comportamento desejado

def get_base_classificacao(id,C):
    time.sleep(3)
    url = f"https://servicodados.ibge.gov.br/api/v3/agregados/{id}/periodos/-30/variaveis?classificacao={C}&localidades=N6[1505536]"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload, timeout=10)
    return response.json()

In [ ]:
def get_base_classificacao(id,C,mun):
    retries = 3  # Número de tentativas
    delay = 2  # Tempo de espera entre tentativas
    for _ in range(retries):
        try:
            url = f"https://servicodados.ibge.gov.br/api/v3/agregados/{id}/periodos/-30/variaveis?classificacao={C}&localidades=N6[{mun}]"
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Lança uma exceção se a resposta indicar um erro HTTP
            return response.json()
        except (requests.RequestException, ValueError) as e:
            print(f"Erro ao fazer a requisição: {e}")
            time.sleep(delay)
            continue
    print(f"Falha nas tentativas de solicitação após várias tentativas. {id}, classificação: {C}")
    return "ERRO"  # Ou levante uma exceção, dependendo do comportamento desejado

In [ ]:
#lista_a_pesquisar = ['200']

In [ ]:
base2 = pd.DataFrame()
for mun in municipios:
    for x in lista_a_pesquisar:
        print(f"Executando Laços na pesquisa: {x}")
        base = get_base(x, mun['codigo_ibge'])
        if(base != "ERRO"):
            for i in base:
                print(i)
                resultados = i.get("resultados", [])
                unidade = i.get("unidade")
                variavel = i.get("variavel")
                id = i.get("id")
                for resultado in resultados:
                    classificacoes = resultado.get("classificacoes", [])
                    series = resultado.get("series", [])
                    if classificacoes:
                        for classificacao in classificacoes:
                            json = get_base_classificacao(x,classificacao.get("id"),mun['codigo_ibge'])
                            for j in json:                        
                                variavel = j.get("variavel")
                                resultados = j.get("resultados", [])
                                unidade = j.get("unidade")
                                id = j.get("id")
                                for resultado in resultados:
                                    series = resultado.get("series", [])
                                    classificacao = resultado.get("classificacoes",[])[0]
                                    classificacao_int = classificacao['nome']
                                    categoria_int = list(classificacao['categoria'].values())[0]
                                    # for classificacao in classificacoes:
                                    #     print(list(classificacao['categoria'].values())[0])
                                    for serie in series:
                                        print(serie.get("serie"))
                                        temp = pd.DataFrame(list(serie.get("serie").items()), columns=['Ano', 'Valor'])
                                        temp['id_agregado'] = x
                                        temp['unidade'] = unidade
                                        temp['variavel'] = variavel
                                        temp['id'] = id
                                        temp['classificacao'] = classificacao_int
                                        temp['categoria'] = categoria_int
                                        temp['municipio'] = mun['municipio']
                                        temp['codigo_ibge'] = mun['codigo_ibge']
                                        base2 = pd.concat([base2,temp])                            
                    else:
                        print("Não tem classificação")
                        for serie in series:
                            print(serie.get("serie"))
                            temp = pd.DataFrame(list(serie.get("serie").items()), columns=['Ano', 'Valor'])
                            temp['id_agregado'] = x
                            temp['unidade'] = unidade
                            temp['variavel'] = variavel
                            temp['id'] = id
                            temp['municipio'] = mun['municipio']
                            temp['codigo_ibge'] = mun['codigo_ibge']
                            base2 = pd.concat([base2,temp])

In [ ]:
base2

In [ ]:
base2.rename(columns={'Ano': 'ano', 'Valor': 'valor'}, inplace=True)

In [ ]:
base = base2.drop_duplicates()

In [ ]:
base

In [ ]:
base['last_update'] = pd.Timestamp.now()

In [ ]:
base.to_csv('IO.csv')

In [ ]:
base

In [ ]:
base.to_sql('ibge_variaveis_pbs_base', schema='pesquisas_ibge', con=conn_hom, if_exists='append', index=False, method='multi', )